In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import CTransformers
from langchain.prompts import PromptTemplate

In [2]:
from llama_index.readers.file import PDFReader

loader = PDFReader()
documents = loader.load_data("medical_book.pdf")  # Just a string, not a list
  # expects a list
print(f"Loaded {len(documents)} pages.")


Loaded 759 pages.


In [3]:
!pip install llama-index-embeddings-huggingface


In [4]:
import importlib.metadata
print(importlib.metadata.version("llama-index"))



0.12.42


In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import ServiceContext


In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# ⚡ Use a fast model instead of BGE
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)



In [7]:
from llama_index.core import Settings

In [9]:
Settings.embed_model = embed_model

NameError: name 'embed_model' is not defined

In [10]:
from llama_index.core import ServiceContext, VectorStoreIndex

In [11]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)




In [12]:
from llama_index.core import Settings
Settings.llm = None  # 🚫 Disable OpenAI fallback globally


LLM is explicitly disabled. Using MockLLM.


In [13]:
query_engine = index.as_query_engine(similarity_top_k=5, llm=None)

In [14]:
!pip install google-generativeai


In [15]:
import google.generativeai as genai

# ⚠️ Set your Gemini API Key
genai.configure(api_key="YOUR_API_KEY")

# Load the Gemini model
gemini_model = genai.GenerativeModel("models/gemini-1.5-flash")


In [16]:
def medical_chatbot(question: str) -> str:
    # Step 1: Retrieve relevant chunks using LlamaIndex
    retrieved_nodes = query_engine.retrieve(question)
    context = "\n\n".join([node.text for node in retrieved_nodes])

    # Step 2: Construct the prompt for Gemini
    prompt = f"""
You are a helpful and knowledgeable medical assistant.
Based on the following medical context, answer the user's question clearly and accurately.

Context:
{context}

Question: {question}
"""

    # Step 3: Send to Gemini
    try:
        response = gemini_model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"❌ Error from Gemini: {e}"


In [17]:
answer = medical_chatbot("What are the symptoms of dengue's disease?")
print(answer)


The provided text states that dengue fever begins with chills, headache, and fever.  Additional symptoms are not explicitly listed in this excerpt.


In [ ]:
answer = medical_chatbot("What is the treatment for depression?")
print("Answer:\n", answer)


In [ ]:
!pip install gradio google-generativeai llama-index sentence-transformers


In [18]:
import gradio as gr

In [19]:
# Gradio UI
iface = gr.Interface(
    fn=medical_chatbot,
    inputs=gr.Textbox(placeholder="Ask a medical question...", lines=3),
    outputs="text",
    title="📘 Medical Chatbot",
    description="Ask questions from the Gale Encyclopedia of Medicine using Gemini + LlamaIndex.",
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
